In [ ]:
# 根据统计数据计算gmm模型。
# 在训练过程中，如果没有clean-clutter图像对，不能计算mask
# 就需要对所有混合的residual进行GMM拟合
# 这份代码把easy和hard像素混合起来，然后转换成每个residual区间有多少个像素。
# 在之前的统计分析发现，统计residual的sum结果对easy和hard区分更明显，转换成像素个数会让hard部分显得不明显，数值比较小。
# 所以这里转换成个数的时候，对大residual的部分进行放大(比如0.5-0.51的residual，除的时候不除以0.505，而是统一除以0.01或者1,反正比例是一样的)
import os
import numpy as np
import matplotlib.pyplot as plt

### easy
# 定义区间宽度
interval_width = 0.01
# 计算区间数量
num_intervals = int(1 / interval_width)

# 初始化区间内元素总和列表
easy_interval_sums = np.zeros(num_intervals)
hard_interval_sums = np.zeros(num_intervals)


step = 20000
# hist_total_file_name = f'training_{step}_steps_residual_statistics_hist_total.npy'
# hist_total = np.load('./' + hist_total_file_name)
easy_interval_sums_file_name = f'easy_training_{step}_steps_residual_statistics_interval_sums.npy'
easy_interval_sums = np.load('./easy/' + easy_interval_sums_file_name)

print(easy_interval_sums.shape)
print(easy_interval_sums.max(), easy_interval_sums.min())
easy_interval_sums

In [ ]:
### hard
# hist_total_file_name = f'training_{step}_steps_residual_statistics_hist_total.npy'
# hist_total = np.load('./' + hist_total_file_name)
hard_interval_sums_file_name = f'hard_training_{step}_steps_residual_statistics_interval_sums.npy'
hard_interval_sums = np.load('./hard/' + hard_interval_sums_file_name)

print(hard_interval_sums.shape)
print(hard_interval_sums.max(), hard_interval_sums.min())
hard_interval_sums

In [ ]:
from sklearn.mixture import GaussianMixture

# 创建一个示例数据集
np.random.seed(0)
X = np.concatenate([np.random.normal(0, 1, 500), np.random.normal(4, 1, 500)]).reshape(-1, 1)
print(X.shape)
# 拟合GMM模型
# gmm = GaussianMixture(n_components=2, random_state=0)
gmm = GaussianMixture(n_components=2, max_iter=10,tol=1e-2,reg_covar=5e-4)
gmm.fit(X)

# 可视化分布曲线
x = np.linspace(-2, 8, 1000)
pdf = np.exp(gmm.score_samples(x.reshape(-1, 1)))
plt.hist(X, bins=30, density=True, alpha=0.5)
plt.plot(x, pdf, '-r')
plt.xlabel('Value')
plt.ylabel('Probability')
plt.show()

In [ ]:
## 方法1：
import numpy as np
from sklearn.mixture import GaussianMixture

# 假设你已经拟合了一个1维、两个components的GMM模型并获得了参数
gmm = GaussianMixture(n_components=2, random_state=0)
gmm.fit(X)  # X 是你的原始数据
means = gmm.means_
covariances = gmm.covariances_
weights = gmm.weights_
print("means:", means)

# 新数据点
new_data = np.array([1.0, 2.0, 3.0]).reshape(-1, 1)

# 计算新数据点的概率密度
num_components = len(means)
num_new_data_points = len(new_data)
pdf_values = np.zeros((num_new_data_points, num_components))

for i in range(num_components):
    # 计算多元正态分布的概率密度
    diff = new_data - means[i]
    cov = covariances[i]  # 协方差矩阵是1x1的
    cov_inv = 1.0 / cov  # 计算协方差矩阵的逆
    exponent = -0.5 * (diff ** 2) / cov  # 修正这一行
    pdf_values[:, i] = ((1.0 / np.sqrt(2 * np.pi * cov)) * np.exp(exponent))[:,0]

# 计算总的概率密度（加权平均）
total_pdf = np.sum(pdf_values * weights, axis=1)

print("概率密度:", total_pdf)
# print("两个概率密度:", pdf_values)
# 计算每个新数据点属于所有分量的概率之和并归一化
normalized_prob_values = (pdf_values * weights)/ total_pdf[:, np.newaxis]
print("归一化概率密度:", normalized_prob_values)
# print("归一化两个概率密度:", pdf_values)

prob = gmm.predict_proba(new_data) 
prob = prob[:,gmm.means_.argmin()] 
print("prob:", prob)

# 使用 GMM 模型的 predict_proba 函数计算概率
# prob = gmm.predict_proba(new_data) 
# prob_min = prob[:, gmm.means_.argmin()] 
# prob_max = prob[:, gmm.means_.argmax()] 
# print("prob:", prob)
# print("prob_min:", prob_min)
# print("prob_max:", prob_max)


In [ ]:
## 方法2：其实和方法1一样，就(1.0 / np.sqrt(2 * np.pi * cov)) * np.exp(exponent)这里 有一点点点点不一样，没有本质区别
import numpy as np
from sklearn.mixture import GaussianMixture

# 假设你已经拟合了一个1维、两个components的GMM模型并获得了参数
gmm = GaussianMixture(n_components=2, random_state=0)
gmm.fit(X)  # X 是你的原始数据
means = gmm.means_
covariances = gmm.covariances_
weights = gmm.weights_

# 新数据点
new_data = np.array([1.0, 2.0, 3.0])

# 计算每个新数据点属于每个分量的概率
num_components = len(means)
num_new_data_points = len(new_data)
prob_values = np.zeros((num_new_data_points, num_components))

for i in range(num_components):
    # 计算多元正态分布的概率密度
    diff = new_data - means[i]
    cov = covariances[i]  # 协方差矩阵是1x1的
    cov_inv = 1.0 / cov  # 计算协方差矩阵的逆
    exponent = -0.5 * (diff ** 2) / cov  # 计算指数部分
    pdf_values = (1.0 / np.sqrt(2 * np.pi * cov)) * np.exp(exponent)
    prob_values[:, i] = pdf_values * weights[i]

# 计算每个新数据点属于所有分量的概率之和并归一化
total_prob = np.sum(prob_values, axis=1)
normalized_prob_values = prob_values / total_prob[:, np.newaxis]

print("每个数据点属于每个分量的概率:\n", normalized_prob_values)

prob = gmm.predict_proba(new_data.reshape(-1, 1)) 
prob = prob[:, gmm.means_.argmin()] 
print("prob:", prob)



In [ ]:
X.shape
X[:,0].shape
X = X[:,0]

In [ ]:
#这样直接实现EM算法，比库函数的计算要偏离很多，从结果可视化可以看到。
import numpy as np
import matplotlib.pyplot as plt

# 假设你的数据集 X 是一维的
# X = np.concatenate([np.random.normal(0, 1, 500), np.random.normal(4, 1, 500)])
print(X.shape)

# 定义GMM参数
num_components = 2  # 分量数量
max_iter = 10  # 最大迭代次数
tolerance = 1e-3  # 容差

# 随机初始化均值、方差和混合系数
np.random.seed(0)
means = np.random.rand(num_components) * 5
std_devs = np.random.rand(num_components)
weights = np.random.rand(num_components)

# 初始化 prev_log_likelihood
prev_log_likelihood = 0

# EM算法估计GMM参数
for _ in range(max_iter):
    # E步：计算每个数据点属于每个分量的概率
    probs = np.zeros((len(X), num_components))
    for i in range(num_components):
        probs[:, i] = weights[i] * (1.0 / (std_devs[i] * np.sqrt(2 * np.pi))) * np.exp(-(X - means[i]) ** 2 / (2 * std_devs[i] ** 2))
    
    probs /= np.sum(probs, axis=1, keepdims=True)
    
    # M步：更新均值、方差和混合系数
    Nk = np.sum(probs, axis=0)
    means = np.sum(X.reshape(-1, 1) * probs, axis=0) / Nk
    std_devs = np.sqrt(np.sum(probs * (X.reshape(-1, 1) - means) ** 2, axis=0) / Nk)
    weights = Nk / len(X)
    
    # 计算对数似然
    log_likelihood = np.sum(np.log(np.sum(probs, axis=1)))
    
    # 判断是否收敛
    if np.abs(log_likelihood - prev_log_likelihood) < tolerance:
        break
    
    prev_log_likelihood = log_likelihood

# 计算模型的概率密度函数
x = np.linspace(-2, 8, 1000)
pdf = np.zeros_like(x)
for i in range(num_components):
    pdf += weights[i] * (1.0 / (std_devs[i] * np.sqrt(2 * np.pi))) * np.exp(-(x - means[i]) ** 2 / (2 * std_devs[i] ** 2))

# 可视化分布曲线
plt.hist(X, bins=30, density=True, alpha=0.5)
plt.plot(x, pdf, '-r')
plt.xlabel('Value')
plt.ylabel('Probability')
plt.show()


In [ ]:
### 把每个区间的residual的求和(sum)转换成数量，ids和ids_2两种转换方式。ids_2更能体现出easy 和 hard的区别。
interval_sums = hard_interval_sums + easy_interval_sums
interval_sums = interval_sums.reshape(-1, 1)
# print((interval_sums.astype(int)[-20:]))

ids = 0.01*(np.array(range(1,101))-0.5).reshape(-1, 1)
# print(ids)

ids_2 = np.ones_like(ids).reshape(-1, 1)
# print(ids_2.shape)

numbers = (interval_sums/ids_2).astype(int)
# print(numbers[:20])

statistic = []
for i, id in enumerate(ids):
    tmp = id * np.ones(numbers[i]).reshape(-1,1)
    statistic.append(tmp)

statistic_all = np.concatenate(statistic)
print(statistic[0].shape)
print(statistic_all.shape)
print(numbers.sum())


In [ ]:

from sklearn.mixture import GaussianMixture

# 拟合GMM模型
# gmm = GaussianMixture(n_components=2, random_state=0)
gmm = GaussianMixture(n_components=2, max_iter=10,tol=1e-2,reg_covar=5e-4)
gmm.fit(statistic_all.reshape(-1, 1))

# 可视化分布曲线
x = np.linspace(0, 1, 1000)
pdf = np.exp(gmm.score_samples(x.reshape(-1, 1)))
plt.hist(statistic_all, bins=100, density=True, alpha=0.5)
plt.plot(x, pdf, '-r')
plt.xlabel('Value')
plt.ylabel('Probability')
plt.show()

In [ ]:
prob = gmm.predict_proba(statistic_all) 
prob = prob[:,gmm.means_.argmin()] 

print(prob.shape)

In [ ]:
### 验证一下用参数计算的概率和库函数的结果是否一致。
means = gmm.means_
covariances = gmm.covariances_
weights = gmm.weights_
# 新数据点
new_data = np.array([0.1, 0.2, 0.5])
new_data = statistic_all[:, 0]

# 计算每个新数据点属于每个分量的概率
num_components = len(means)
num_new_data_points = len(new_data)
prob_values = np.zeros((num_new_data_points, num_components))

for i in range(num_components):
    # 计算多元正态分布的概率密度
    diff = new_data - means[i]
    cov = covariances[i]  # 协方差矩阵是1x1的
    cov_inv = 1.0 / cov  # 计算协方差矩阵的逆
    exponent = -0.5 * (diff ** 2) / cov  # 计算指数部分
    pdf_values = (1.0 / np.sqrt(2 * np.pi * cov)) * np.exp(exponent)
    prob_values[:, i] = pdf_values * weights[i]

# 计算每个新数据点属于所有分量的概率之和并归一化
total_prob = np.sum(prob_values, axis=1)
normalized_prob_values = prob_values / total_prob[:, np.newaxis]
print("gmm.means_:", gmm.means_)
print("gmm.means_.argmin():", gmm.means_.argmin())
print("每个数据点属于每个分量的概率:\n", normalized_prob_values)

prob = gmm.predict_proba(new_data.reshape(-1, 1)) 
prob = prob[:, gmm.means_.argmin()] 
print("prob:", prob)

prob_my = normalized_prob_values
prob_my = prob_my[:, gmm.means_.argmin()] 
print("prob_my:", prob_my)

error = prob-prob_my
print(error.max())

